(sec:coord)=

# Choice of coordinates

A fundamental concept in quantum chemistry is the multi-dimensional potential energy surface (PES). It captures the interplay between the electronic and nuclear degrees of freedom in a system, and stationary points on the PES are important in elucidating molecular conformations and explaining the mechanisms of chemical and photo-chemical reactions. The first- (gradient) and second-order (Hessian) derivatives of the energy with respect to nuclear displacements are key elements in locating these points (minima and transition states) and determining minimum energy reaction pathways. 

We will here discuss the choice of coordinates in which these energy derivatives are expressed.

```{figure} ../../img/pes/coordinates.png
---
name: coordinates
width: 600px
align: center
---
```

## Cartesian coordinates

The most straightforward way to define atomic positions is to use a Cartesian reference system and define each atomic position in terms of its $(x,y,z)$-coordinates. In this coordinate system it is very easy to determine the total energy and energy gradient, but this choice is not favorable for geometry optimization because these coordinates are strongly coupled to one another. 

## Internal coordinates

A more favorable choice is to work with internal coordinates, such as bond lengths, valence angles and dihedrals. The most well known set of internal coordinates is the Z-matrix, which uses this properties in order to describe the molecular structure. However, using internal coordinates poses two challenges: (1) the choice of coordinates is not unique, and (2) internal coordinates have to be transformed back into Cartesian coordinates to compute the energy and gradient. Several ways of handling these problems are discussed below.

(sec:internal-to-cartesian)=
## Transforming between coordinate systems

%One common choice of coordinates in molecular structure optimization are the *redundant internal coordinates* (RIC) {cite}`Pulay1992`.

Given the set of internal coordinates $\mathbf{q}=(q_1,q_2,...,q_{n_q})$, we would like to relate displacements performed in these coordinates to displacements in the $n_x=3N$ Cartesian coordinates $\mathbf{x}=(x_1,x_2,...,x_{n_x})$ (to simplify, we denoted all Cartesian coordinates by $x$). For this purpose, we define matrix $\mathbf{B}$ (called the Wilson $\mathbf{B}$-matrix) {cite}`orcamanual`

$$
    B_{ij} = \frac{\partial q_i}{\partial x_j}
$$

To determine the changes in $\mathbf{x}$ as a result of changes performed in $\mathbf{q}$, we basically need to obtain the inverse relation $\partial x_j/\partial q_i$. However, because $\mathbf{B}$ is generally rectangular and cannot be directly inverted, we first have to construct a square matrix $\mathbf{G}$

$$
    \mathbf{G} = \mathbf{B}\mathbf{B}^\mathrm{T} 
$$

If we are using non-redundant internal coordinates, $\mathbf{G}$ can be inverted directly. However, if the internal coordinates are redundant, some of the rows of $\mathbf{G}$ will be linearly dependent. In this case, we have to set up and solve an eigenvalue equation that will allow us to separate out the redundancies {cite}`Pulay1992`

$$
    \mathbf{G} 
    \begin{pmatrix}
    \mathbf{U} & \mathbf{R}
    \end{pmatrix} = 
    \begin{pmatrix}
    \mathbf{U} & \mathbf{R}
    \end{pmatrix} 
    \begin{bmatrix}
    \boldsymbol{\Lambda} & 0 \\
    0 & 0
    \end{bmatrix} 
$$

This equation has $n=3N-6$ (or $3N-5$ for linear molecules) non-zero eigenvalues and $n_q-n$ zero eigenvalues. The non-zero eigenvalues correspond to linearly independent coordinates, while those which are zero identify the redundant ones. Furthermore, the first $n$ eigenvectors contained in the matrix $\mathbf{U}$ are non-redundant and can be used to define a set of non-redundant coordinates $\mathbf{s}$:

(eq:coord_nonred)=
$$
    \mathbf{s} = \mathbf{U}^\mathrm{T} \mathbf{q} 
$$

The eigenvalue equation can be used to define a generalized inverse matrix $\mathbf{G}^{-}$:

$$
 \mathbf{G}^{-} = 
    \begin{pmatrix}
    \mathbf{U} & \mathbf{R}
    \end{pmatrix}    
    \begin{bmatrix}
    \boldsymbol{\Lambda}^{-1} & 0 \\
    0\,\,\,\,\,\,\,\, & 0
    \end{bmatrix}
    \begin{pmatrix}
    \mathbf{U}^\mathrm{T}\\ 
    \mathbf{R}^\mathrm{T}
    \end{pmatrix}
$$

which, in turn, is used to determine the transformation between Cartesian and internal coordinates displacements {cite}`Wang2016`:

$$
    \Delta\mathbf{x} = \mathbf{B}^\mathrm{T}\mathbf{G}^{-}\Delta\mathbf{q}
$$ 

In a similar way, the gradient can be transformed from Cartesian to internal coordinates {cite}`orcamanual`:

$$
    \mathbf{g}_q = \mathbf{G}^{-}\mathbf{B}\mathbf{g}_x 
$$

where we have denoted the gradient in Cartesian coordinates by $\mathbf{g}_x$ and the gradient in internal coordinates by $\mathbf{g}_q$. 

With these equations we can now transform a displacement in internal coordinates to a displacement in Cartesian coordinates, compute the energy gradient, and transform the gradient back to internal coordinates.

In the case of the Hessian matrix, the transformation to internal coordinates requires the second-order derivatives of $q_i$ with respect to the Cartesian coordinates $x_j$, $x_k$

$$
    B^{(2)}_{ijk} = \frac{\partial^2 q_i}{\partial x_j\partial x_k}
$$

These derivatives, together with $\mathbf{G}^{-}$, the Wilson $\mathbf{B}$-matrix and gradient are then used to transform the Hessian from Cartesian ($\mathbf{H}_x$) to internal ($\mathbf{H}_q$) coordinates

$$
    \mathbf{H}_q = \mathbf{G}^{-} \mathbf{B} \left[ \mathbf{H}_x - \mathbf{g}_q \mathbf{B}^{(2)}\right]\mathbf{B}^\mathrm{T} \mathbf{G}^{-\mathrm{T}}
$$

After these transformations are carried out, further transformations to related internal coordinates, for example to use $1/R$ instead of $R$ (bond length), are easy to implement, as exemplified for the gradient: Given the gradient in terms of $R$

$$
    g(R) = \frac{\mathrm{d} E}{\mathrm{d}R}
$$

the transformation to $u=1/R$ is a simple change of variables

$$
    g(u) = \frac{\mathrm{d} E}{\mathrm{d}u}=\frac{\mathrm{d} E}{\mathrm{d}R}\frac{\mathrm{d} R}{\mathrm{d}u} = \frac{\mathrm{d} E}{\mathrm{d}R}\frac{\mathrm{d} u^{-1}}{\mathrm{d}u} =  -u^{-2}\,g(R) =  -R^{2} \,g(R)
$$

The transformation of the Hessian can be carried out similarly.


<!-- ## Delocalized internal coordinates

Geometry optimizations using internal coordinates require, as we have seen above, to find the generalized inverse matrix $\mathbf{G}^-$. This is an expensive step, since the matrices involved are constructed in terms of the primitive internal coordinates, which are very numerous. One idea to reduce this computational cost is to use the [non-redundant coordinates](eq:coord_nonred). These new coordinates are actually linear combinations of primitives, and therefore are called "delocalized" internal coordinates (DLC) {cite}`Baker1999`.

Within this coordinate system, we define a new $\mathbf{B}$ matrix which now has a significantly smaller dimension (3$N$-6) compared to the $\mathbf{B}$ matrix in primitive internal coordinates. The new $\mathbf{B}$ matrix is then used to reformulate the displacement and gradient transformations between the Cartesian and delocalized internal coordinate systems. For a detailed derivation of these transformations see {cite}`Baker1999`.


### Hybrid delocalized internal coordinates

The situation becomes a bit more complicated when the system we would like to optimize is composed of two or more individual molecules. In this case, fictitious intermolecular bonds, angles, and dihedrals have to be included to define the relative positions between the separate components of the multi-molecular system {cite}`Billeter2000, Wang2016`. In an analogous way as for single molecules, hybrid delocalized internal coordinates (HDLC) can be obtained by including the intermolecular primitives in the non-redundant transformation. In this approach, the intra- and intermolecular degrees of freedom become mixed, *i.e.*, a displacement in one intermolecular primitive will affects both the intramolecular and intermolecular structures {cite}`Wang2016`.


## Translation--rotation internal coordinates

Another option is to treat the intra- and intermolecular coordinates separately, by introducing translation and rotation coordinates into the set of internal primitives. The resulting choice of coordinates are known as translation--rotation internal coordinates (TRIC) {cite}`Wang2016`. What this actually means is that, instead of adding the full set of Cartesian coordinates in the construction of primitive internal coordinates (HDLC), only six new internal coordinates are used for each molecule to describe its translation and rotation, essentially constraining the intermolecular orientations. This reduces the computational cost and represents a more intuitive way to define the relative orientation between molecules {cite}`Wang2016`.
 -->